In [38]:
import pandas as pd
import re
import os
from datetime import datetime
from urllib import request

### Récupération des fichiers CSV de l'état
Les fichiers sont récupérés ici : https://files.data.gouv.fr/geo-dvf/latest/csv/


In [39]:
years = [2018, 2019, 2020, 2021, 2022, 2023]

df = pd.DataFrame()

for year in years:
    print(f"Downloading {year}")
    url = f"https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/full.csv.gz"
    year_df = pd.read_csv(url, compression='gzip', low_memory=False)
    df = pd.concat([df, year_df])

### Nettoyage des données
On va supprimer toutes les lignes qui ne sont pas utiles

In [40]:
cleaned_df = df.drop(['id_mutation','numero_disposition', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'code_commune', 'code_departement', 'ancien_code_commune', 'adresse_suffixe', 'ancien_nom_commune', 'nature_culture', 'nature_culture_speciale','nature_mutation', 'code_nature_culture_speciale', 'code_nature_culture','lot1_numero','lot1_surface_carrez','lot2_numero','lot2_surface_carrez','lot3_numero', 'lot3_surface_carrez','lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez'], axis=1)

### Renommer les colonnes
Les colonnes doivent avoir le même nom que les colonnes dans la notice fournit ici : https://www.data.gouv.fr/fr/datasets/r/d573456c-76eb-4276-b91c-e6b9c89d6656

In [41]:
pdf_url = "https://www.data.gouv.fr/fr/datasets/r/d573456c-76eb-4276-b91c-e6b9c89d6656"
request.urlretrieve(pdf_url, os.path.join(os.path.abspath('..'), 'ressources', 'notice_dvf.pdf'))


cleaned_df['Date de mutation'] = pd.to_datetime(cleaned_df['date_mutation'])
cleaned_df['Annee'] = cleaned_df['Date de mutation'].map(lambda x: x.year)
cleaned_df['Jour'] = cleaned_df['Date de mutation'].map(lambda x: x.day)
cleaned_df['Mois'] = cleaned_df['Date de mutation'].map(lambda x: x.month)
cleaned_df['Column1'] = datetime.now()
cleaned_df = cleaned_df.rename(columns={'valeur_fonciere': 'Valeur foncière', 'adresse_numero': 'N° de voie', 'adresse_nom_voie':'Voie', 'adresse_code_voie':'Code voie', 'code_postal': 'Code postal', 'nom_commune': 'Commune', 'nombre_lots': 'Nombre de lots', 'code_type_local': 'Code type local', 'type_local': 'Type local', 'surface_reelle_bati': 'Surface réelle bâti', 'nombre_pieces_principales': 'Nombre de pièces principales', 'surface_terrain': 'Surface terrain'})


### Harmonisation des données
Rendre "Commune" plus simple à lire en mettant tout en majuscule, séparer l'arrondissement et le nom de commune

In [42]:
cleaned_df = cleaned_df.drop(['date_mutation'], axis=1)
cleaned_df['Commune'] = cleaned_df['Commune'].str.upper()
cleaned_df = cleaned_df[(cleaned_df['Type local'] == 'Maison') | (cleaned_df['Type local'] == 'Appartement')]

In [43]:
pattern = r"(\D+)\s(\d+(?:E|ER))\sARRONDISSEMENT"
number_pattern = r"(E|ER)$"

def extract_commune_arrondissement(texte):
    match = re.match(pattern, texte)
    if match:
        return match.group(1).strip(), re.sub(number_pattern, "", match.group(2))
    else:
        return texte, None

In [44]:
cleaned_df[['Commune', 'Arrondissement']] = cleaned_df['Commune'].apply(
    lambda x: extract_commune_arrondissement(x)
).tolist()

### Sauvegarde des données

In [45]:
cleaned_df.to_csv(os.path.join(os.path.abspath('..'), 'ressources', 'french_real_estate_prices.csv'), index=False)